In [1]:
import sys
sys.path.insert(0, 'env')
sys.path.insert(0, 'lib')

import ccg
from agents import ARagent, A2Cagent, SkipAgent
import pandas as pd
import numpy as np
import copy
import random
import torch
import utils
seed = 1231432

np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)

cards = pd.read_csv('env/configue/cardsTable.csv')
cardsList = [ccg.Minion(i) for i in cards.values.tolist()]

cores = pd.read_csv('env/configue/cores.csv')
coreList = [ccg.Core(i) for i in cores.values.tolist()]

playersNum = 2
piles_player = [ccg.Pile(cardsList, 10) for _ in range(4)]
piles = [copy.deepcopy(piles_player) for _ in range(playersNum)]
cores = np.random.choice(coreList, 2)
decks = [ccg.Deck(cores[i], piles[i], i) for i in range(playersNum)]

In [2]:
cards

,keyName,type,damage,armour,maxHealth,cost,priority,charge,baseActivations
0,HS_PUDDLESTOMPER,minion,3,0,2,2,0,0,1
1,HS_RIVER_CROCOLISK,minion,2,0,3,2,0,0,1
2,HS_THRALLMAR_FARSEER,minion,2,0,3,3,0,0,2
3,HS_DUSKBOAR,minion,4,0,1,2,0,0,1


In [3]:
session = ccg.Session(cardsList, coreList, 2)

In [4]:
from __future__ import print_function
print(torch.__version__)

0.4.1


In [5]:
from trainer import Trainer
from agents import ARagent, A2Cagent, Root
from nets import ActorNetwork, ValueNetwork
from replays import FlatReplay, PrioritizedReplay

root = Root(session)

In [6]:
def plan_mcts(root,n_iters=10):
    """
    builds tree with monte-carlo tree search for n_iters iterations
    :param root: tree node to plan from
    :param n_iters: how many select-expand-simulate-propagete loops to make
    """
    for _ in range(n_iters):

        node = root.select_best_leaf()

        if node.is_done:
            node.propagate(0)

        else:
            node.expand()
            rew = node.rollout()
            node.propagate(rew)
    
plan_mcts(root,n_iters=1000)

In [8]:
from IPython.display import clear_output
from itertools import count
from gym.wrappers import Monitor

total_reward = 0                #sum of rewards
test_env = loads(root_snapshot) #env used to show progress
#print(count())
for i in count():
    #print(i)
    #get best child
    children = root.children
    m = None
    best_child = None
    for i in children:
        if(m == None):
            best_child = i
            m = i.ucb_score() 
        elif (i.ucb_score() > m):
            best_child = i
            m = i.ucb_score() 
    
    #take action
    s,r,done,_ = test_env.step(best_child.action)
    
    #show image
    clear_output(True)
    #print(i)
    #plt.title("step %i"%i)
    plt.imshow(test_env.render('rgb_array'))
    plt.show()

    total_reward += r
    if done:
        print("Finished with reward = ",total_reward)
        break
    
    #discard unrealized part of the tree [because not every child matters :(]
    for child in root.children:
        if child != best_child:
            child.safe_delete()

    #declare best child a new root
    root = Root.from_node(best_child)
    
    if(root.is_leaf()):
        plan_mcts(root)
    
    assert not root.is_leaf(), "We ran out of tree! Need more planning! Try growing tree right inside the loop."
    
    #you may want to expand tree here
    #<your code here>


NameError: name 'loads' is not defined